In [1]:
import numpy as np
import cv2

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

from srgan import SRGAN
from generator import ImageDataGenerator

Using TensorFlow backend.


In [2]:
!TF_ENABLE_GPU_GARBAGE_COLLECTION=false

In [3]:
# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()

In [4]:
input_dir = 'data/train/low/'
tgt_dir = 'data/train/high/'

test_file = 'UBMk30rjy0o_17675_1.jpg'
test_low_path =  'data/val/low/' + test_file
test_high_path = 'data/val/high/' + test_file
test_out_path = 'data/out/' + test_file
test_test_path = 'data/test/' + test_file

vgg_path = 'vgg16_notop.hdf5'

input_shape = (135, 240, 3)
tgt_shape = (540, 960, 3)

batch_size = 1

In [5]:
srgan = SRGAN(vgg_path, batch_size, input_shape, tgt_shape)
gen = ImageDataGenerator(input_dir, tgt_dir, batch_size)

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [6]:
test_img_low = ((cv2.imread(test_low_path) - 127.5) / 127.5)[np.newaxis, :, :, :]
test_img_high = ((cv2.imread(test_high_path)- 127.5) / 127.5)[np.newaxis, :, :, :]

test_img_sr = srgan.generator.predict(test_img_low)
test_img_sr_int = ((test_img_sr * 127.5) + 127.5).astype('u1')[0]
cv2.imwrite(test_test_path, test_img_sr_int)

True

In [7]:
for e in range(10000):
    print('epoch', e)
    X_low, X_high = gen.flow_from_directory().__next__()
    srgan.train(X_low, X_high)
    if e % 1000 == 0:
        test_img_low = ((cv2.imread(test_low_path) - 127.5) / 127.5)[np.newaxis, :, :, :]
        test_img_high = ((cv2.imread(test_high_path)- 127.5) / 127.5)[np.newaxis, :, :, :]

        test_img_sr = srgan.generator.predict(test_img_low)
        test_img_sr_int = ((test_img_sr * 127.5) + 127.5).astype('u1')[0]
        test_out_path = 'data/out/'  + test_file.split('.')[0] + f'_test_{e}.jpg'
        cv2.imwrite(test_out_path, test_img_sr_int)

epoch 0
d_loss_real:0.539212167263031, d_loss_fake:0.9005712270736694, vgg_loss:52.800498962402344
epoch 1
d_loss_real:0.5304208993911743, d_loss_fake:0.8944519758224487, vgg_loss:100.36981964111328
epoch 2
d_loss_real:0.5351473689079285, d_loss_fake:0.8858605623245239, vgg_loss:24.411256790161133
epoch 3
d_loss_real:0.541367769241333, d_loss_fake:0.8768856525421143, vgg_loss:45.60787582397461
epoch 4
d_loss_real:0.547719419002533, d_loss_fake:0.8682332038879395, vgg_loss:150.05780029296875
epoch 5
d_loss_real:0.5539023280143738, d_loss_fake:0.8600367307662964, vgg_loss:71.39922332763672
epoch 6
d_loss_real:0.5598636865615845, d_loss_fake:0.8522766828536987, vgg_loss:45.87934494018555
epoch 7
d_loss_real:0.5656038522720337, d_loss_fake:0.8449180126190186, vgg_loss:62.564029693603516
epoch 8
d_loss_real:0.5711328387260437, d_loss_fake:0.8379054069519043, vgg_loss:37.6025390625
epoch 9
d_loss_real:0.57645583152771, d_loss_fake:0.8312146067619324, vgg_loss:137.96139526367188
epoch 10
d_lo

d_loss_real:0.6913446187973022, d_loss_fake:0.7024499773979187, vgg_loss:30.93121910095215
epoch 83
d_loss_real:0.6915635466575623, d_loss_fake:0.7022280693054199, vgg_loss:25.569690704345703
epoch 84
d_loss_real:0.6917732954025269, d_loss_fake:0.7020143866539001, vgg_loss:16.758577346801758
epoch 85
d_loss_real:0.6919745206832886, d_loss_fake:0.7018084526062012, vgg_loss:14.303980827331543
epoch 86
d_loss_real:0.6921675801277161, d_loss_fake:0.7016109228134155, vgg_loss:40.16609573364258
epoch 87
d_loss_real:0.6923530101776123, d_loss_fake:0.7014225125312805, vgg_loss:22.111244201660156
epoch 88
d_loss_real:0.6925312876701355, d_loss_fake:0.7012422680854797, vgg_loss:16.62082290649414
epoch 89
d_loss_real:0.6927027106285095, d_loss_fake:0.7010693550109863, vgg_loss:50.135101318359375
epoch 90
d_loss_real:0.6928673982620239, d_loss_fake:0.7009027004241943, vgg_loss:19.47361183166504
epoch 91
d_loss_real:0.6930258870124817, d_loss_fake:0.7007428407669067, vgg_loss:105.99746704101562
epo

d_loss_real:0.6966092586517334, d_loss_fake:0.6970643997192383, vgg_loss:21.82149887084961
epoch 165
d_loss_real:0.6966164112091064, d_loss_fake:0.6970553994178772, vgg_loss:20.693222045898438
epoch 166
d_loss_real:0.6966232657432556, d_loss_fake:0.6970463395118713, vgg_loss:18.222169876098633
epoch 167
d_loss_real:0.69663006067276, d_loss_fake:0.6970376372337341, vgg_loss:29.4913330078125
epoch 168
d_loss_real:0.6966366171836853, d_loss_fake:0.6970293521881104, vgg_loss:28.512256622314453
epoch 169
d_loss_real:0.6966431140899658, d_loss_fake:0.6970216035842896, vgg_loss:11.830437660217285
epoch 170
d_loss_real:0.6966492533683777, d_loss_fake:0.6970141530036926, vgg_loss:12.537736892700195
epoch 171
d_loss_real:0.6966548562049866, d_loss_fake:0.6970067024230957, vgg_loss:12.979451179504395
epoch 172
d_loss_real:0.6966601014137268, d_loss_fake:0.696999728679657, vgg_loss:5.245815753936768
epoch 173
d_loss_real:0.6966649293899536, d_loss_fake:0.6969929933547974, vgg_loss:12.2426576614379

d_loss_real:0.6967692375183105, d_loss_fake:0.6967855095863342, vgg_loss:53.894264221191406
epoch 247
d_loss_real:0.6967687010765076, d_loss_fake:0.696784257888794, vgg_loss:13.560219764709473
epoch 248
d_loss_real:0.6967679858207703, d_loss_fake:0.6967828869819641, vgg_loss:9.147913932800293
epoch 249
d_loss_real:0.696767270565033, d_loss_fake:0.6967813968658447, vgg_loss:29.310081481933594
epoch 250
d_loss_real:0.6967665553092957, d_loss_fake:0.6967800855636597, vgg_loss:27.88275909423828
epoch 251
d_loss_real:0.6967659592628479, d_loss_fake:0.6967787742614746, vgg_loss:14.704111099243164
epoch 252
d_loss_real:0.6967658400535583, d_loss_fake:0.6967777609825134, vgg_loss:10.135024070739746
epoch 253
d_loss_real:0.6967658400535583, d_loss_fake:0.696776807308197, vgg_loss:21.148075103759766
epoch 254
d_loss_real:0.6967659592628479, d_loss_fake:0.6967759132385254, vgg_loss:27.34113883972168
epoch 255
d_loss_real:0.6967660188674927, d_loss_fake:0.6967750787734985, vgg_loss:19.048294067382

d_loss_real:0.6967343688011169, d_loss_fake:0.6967037916183472, vgg_loss:17.613323211669922
epoch 328
d_loss_real:0.6967337131500244, d_loss_fake:0.6967028975486755, vgg_loss:25.675125122070312
epoch 329
d_loss_real:0.6967330574989319, d_loss_fake:0.6967021226882935, vgg_loss:67.45358276367188
epoch 330
d_loss_real:0.6967326402664185, d_loss_fake:0.6967014074325562, vgg_loss:25.461427688598633
epoch 331
d_loss_real:0.696732223033905, d_loss_fake:0.6967005729675293, vgg_loss:20.18560218811035
epoch 332
d_loss_real:0.6967318654060364, d_loss_fake:0.6966999173164368, vgg_loss:94.28434753417969
epoch 333
d_loss_real:0.696731448173523, d_loss_fake:0.6966993808746338, vgg_loss:72.54145050048828
epoch 334
d_loss_real:0.6967311501502991, d_loss_fake:0.6966989040374756, vgg_loss:19.217273712158203
epoch 335
d_loss_real:0.6967307329177856, d_loss_fake:0.6966984272003174, vgg_loss:33.2830696105957
epoch 336
d_loss_real:0.6967304944992065, d_loss_fake:0.6966978311538696, vgg_loss:23.54929351806640

...

d_loss_real:0.6960815191268921, d_loss_fake:0.6959908604621887, vgg_loss:9.987560272216797
epoch 9645
d_loss_real:0.6960813999176025, d_loss_fake:0.6959905028343201, vgg_loss:8.924118995666504
epoch 9646
d_loss_real:0.6960814595222473, d_loss_fake:0.6959906220436096, vgg_loss:7.076778888702393
epoch 9647
d_loss_real:0.6960813403129578, d_loss_fake:0.6959903240203857, vgg_loss:18.71588897705078
epoch 9648
d_loss_real:0.6960813999176025, d_loss_fake:0.6959903240203857, vgg_loss:4.058347702026367
epoch 9649
d_loss_real:0.6960812211036682, d_loss_fake:0.6959900856018066, vgg_loss:12.149908065795898
epoch 9650
d_loss_real:0.6960811614990234, d_loss_fake:0.6959900259971619, vgg_loss:3.469574213027954
epoch 9651
d_loss_real:0.6960810422897339, d_loss_fake:0.695989727973938, vgg_loss:0.9674440622329712
epoch 9652
d_loss_real:0.6960809230804443, d_loss_fake:0.695989727973938, vgg_loss:19.490251541137695
epoch 9653
d_loss_real:0.69608074426651, d_loss_fake:0.6959895491600037, vgg_loss:12.0250501

d_loss_real:0.696077823638916, d_loss_fake:0.6959867477416992, vgg_loss:13.450071334838867
epoch 9726
d_loss_real:0.6960779428482056, d_loss_fake:0.6959867477416992, vgg_loss:5.925705432891846
epoch 9727
d_loss_real:0.6960778832435608, d_loss_fake:0.6959866285324097, vgg_loss:15.767404556274414
epoch 9728
d_loss_real:0.6960778832435608, d_loss_fake:0.6959867477416992, vgg_loss:3.271139621734619
epoch 9729
d_loss_real:0.6960778832435608, d_loss_fake:0.6959867477416992, vgg_loss:17.625879287719727
epoch 9730
d_loss_real:0.6960780024528503, d_loss_fake:0.6959869265556335, vgg_loss:5.870692253112793
epoch 9731
d_loss_real:0.6960778832435608, d_loss_fake:0.6959868669509888, vgg_loss:5.733485221862793
epoch 9732
d_loss_real:0.6960780024528503, d_loss_fake:0.6959870457649231, vgg_loss:24.17618751525879
epoch 9733
d_loss_real:0.6960779428482056, d_loss_fake:0.6959871649742126, vgg_loss:1.8909252882003784
epoch 9734
d_loss_real:0.6960779428482056, d_loss_fake:0.6959874033927917, vgg_loss:8.4020

d_loss_real:0.696076512336731, d_loss_fake:0.6959856748580933, vgg_loss:2.211439847946167
epoch 9807
d_loss_real:0.696076512336731, d_loss_fake:0.6959856748580933, vgg_loss:39.357383728027344
epoch 9808
d_loss_real:0.6960766315460205, d_loss_fake:0.695985734462738, vgg_loss:6.010223388671875
epoch 9809
d_loss_real:0.6960766315460205, d_loss_fake:0.6959856748580933, vgg_loss:3.7326157093048096
epoch 9810
d_loss_real:0.6960767507553101, d_loss_fake:0.6959857940673828, vgg_loss:8.073481559753418
epoch 9811
d_loss_real:0.6960766315460205, d_loss_fake:0.6959856748580933, vgg_loss:2.1130993366241455
epoch 9812
d_loss_real:0.6960767507553101, d_loss_fake:0.6959857940673828, vgg_loss:8.096972465515137
epoch 9813
d_loss_real:0.6960766911506653, d_loss_fake:0.6959855556488037, vgg_loss:5.23319149017334
epoch 9814
d_loss_real:0.6960768103599548, d_loss_fake:0.6959856152534485, vgg_loss:7.735762596130371
epoch 9815
d_loss_real:0.6960767507553101, d_loss_fake:0.6959854364395142, vgg_loss:6.90932226

d_loss_real:0.6960744261741638, d_loss_fake:0.6959821581840515, vgg_loss:9.701364517211914
epoch 9888
d_loss_real:0.6960744857788086, d_loss_fake:0.6959821581840515, vgg_loss:9.202614784240723
epoch 9889
d_loss_real:0.696074366569519, d_loss_fake:0.6959819197654724, vgg_loss:49.018211364746094
epoch 9890
d_loss_real:0.6960744857788086, d_loss_fake:0.6959819793701172, vgg_loss:9.986235618591309
epoch 9891
d_loss_real:0.696074366569519, d_loss_fake:0.6959818601608276, vgg_loss:12.661752700805664
epoch 9892
d_loss_real:0.6960744261741638, d_loss_fake:0.6959819197654724, vgg_loss:11.768651008605957
epoch 9893
d_loss_real:0.696074366569519, d_loss_fake:0.6959817409515381, vgg_loss:5.307061195373535
epoch 9894
d_loss_real:0.696074366569519, d_loss_fake:0.6959818005561829, vgg_loss:7.926368236541748
epoch 9895
d_loss_real:0.6960743069648743, d_loss_fake:0.6959816217422485, vgg_loss:7.239895343780518
epoch 9896
d_loss_real:0.6960743069648743, d_loss_fake:0.6959818005561829, vgg_loss:4.09313106

d_loss_real:0.6960720419883728, d_loss_fake:0.6959794759750366, vgg_loss:16.77589225769043
epoch 9969
d_loss_real:0.696071982383728, d_loss_fake:0.6959793567657471, vgg_loss:4.6182475090026855
epoch 9970
d_loss_real:0.6960720419883728, d_loss_fake:0.6959794759750366, vgg_loss:4.2932353019714355
epoch 9971
d_loss_real:0.6960720419883728, d_loss_fake:0.6959793567657471, vgg_loss:9.029488563537598
epoch 9972
d_loss_real:0.6960721611976624, d_loss_fake:0.6959793567657471, vgg_loss:58.60662841796875
epoch 9973
d_loss_real:0.6960722208023071, d_loss_fake:0.6959792375564575, vgg_loss:2.2153079509735107
epoch 9974
d_loss_real:0.6960722804069519, d_loss_fake:0.6959792971611023, vgg_loss:11.22439193725586
epoch 9975
d_loss_real:0.6960722208023071, d_loss_fake:0.6959792375564575, vgg_loss:19.2332706451416
epoch 9976
d_loss_real:0.6960724592208862, d_loss_fake:0.6959793567657471, vgg_loss:8.963434219360352
epoch 9977
d_loss_real:0.6960723996162415, d_loss_fake:0.6959792375564575, vgg_loss:5.900528

In [8]:
test_img_low = ((cv2.imread(test_low_path) - 127.5) / 127.5)[np.newaxis, :, :, :]
test_img_high = ((cv2.imread(test_high_path)- 127.5) / 127.5)[np.newaxis, :, :, :]

test_img_sr = srgan.generator.predict(test_img_low)
test_img_sr_int = ((test_img_sr * 127.5) + 127.5).astype('u1')[0]
cv2.imwrite(test_out_path, test_img_sr_int)

True